# Credit Risk Modeling (LGD and EAD Model)

### - We create a beta regression model to predict loss given default (LGD) and exposure at default (EAD) for a customer.

# 

## 1) Import Libraries and Dataset

In [1]:
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['agg.path.chunksize'] = 10000
import seaborn as sns
sns.set_theme(style="white", palette='pastel')
%matplotlib inline
pd.options.display.max_columns = None

In [2]:
df_backup = pd.read_csv('loan_data_2007_2014.csv')

In [3]:
df = df_backup.iloc[:,1:].copy()

In [4]:
df.head(3)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000,5000,4975.00,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.00,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.00,Jan-85,1.00,nan,nan,3.00,0.00,13648,83.70,9.00,f,0.00,0.00,5861.07,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,NaN,Jan-16,0.00,nan,1,INDIVIDUAL,nan,nan,nan,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1077430,1314167,2500,2500,2500.00,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.00,Source Verified,Dec-11,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.00,Apr-99,5.00,nan,nan,3.00,0.00,1687,9.40,4.00,f,0.00,0.00,1008.71,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,NaN,Sep-13,0.00,nan,1,INDIVIDUAL,nan,nan,nan,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,1077175,1313524,2400,2400,2400.00,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.00,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.00,Nov-01,2.00,nan,nan,2.00,0.00,2956,98.50,10.00,f,0.00,0.00,3003.65,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,NaN,Jan-16,0.00,nan,1,INDIVIDUAL,nan,nan,nan,0.00,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [5]:
#sorted(df.columns.values)

In [6]:
del_columns = 'all_util, annual_inc_joint, application_type, collection_recovery_fee, collections_12_mths_ex_med, desc, '\
             +'dti_joint, emp_title, funded_amnt_inv, id, il_util, inq_fi, inq_last_12m, last_credit_pull_d, last_pymnt_amnt, '\
             +'last_pymnt_d, loan_amnt, max_bal_bc, member_id, mths_since_last_delinq, mths_since_last_major_derog, '\
             +'mths_since_last_record, mths_since_rcnt_il, next_pymnt_d, open_acc_6m, open_il_12m, open_il_24m, open_il_6m, '\
             +'open_rv_12m, open_rv_24m, out_prncp_inv, policy_code, revol_bal, revol_util, title, tot_coll_amt, '\
             +'tot_cur_bal, total_bal_il, total_cu_tl, total_pymnt_inv, total_rec_late_fee, url, verification_status_joint, '\
             +'zip_code, sub_grade, total_rev_hi_lim'

In [7]:
df.drop(del_columns.split(', '), axis=1, inplace=True)
df.head(3)

,funded_amnt,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq
0,5000,36 months,10.65,162.87,B,10+ years,RENT,24000.00,Verified,Dec-11,Fully Paid,n,credit_card,AZ,27.65,0.00,Jan-85,1.00,3.00,0.00,9.00,f,0.00,5861.07,5000.00,861.07,0.00,0.00
1,2500,60 months,15.27,59.83,C,< 1 year,RENT,30000.00,Source Verified,Dec-11,Charged Off,n,car,GA,1.00,0.00,Apr-99,5.00,3.00,0.00,4.00,f,0.00,1008.71,456.46,435.17,117.08,0.00
2,2400,36 months,15.96,84.33,C,10+ years,RENT,12252.00,Not Verified,Dec-11,Fully Paid,n,small_business,IL,8.72,0.00,Nov-01,2.00,2.00,0.00,10.00,f,0.00,3003.65,2400.00,603.65,0.00,0.00


In [8]:
#df.info()

# 

## 2) Data cleaning

### 2.a. Change type from object to numeric for following columns:<br><br>term, emp_length, earliest_cr_line, issue_date

In [9]:
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [10]:
df['term_int'] = df['term'].str[1:3]
df['term_int'] = pd.to_numeric(df['term_int'])
df['term_int'].unique()

array([36, 60], dtype=int64)

In [11]:
df['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [12]:
df['emp_length_int'] = df['emp_length'].str.replace('\+ years', '')
df['emp_length_int'] = df['emp_length_int'].str.replace('< 1 year', str(0))
df['emp_length_int'] = df['emp_length_int'].str.replace('n/a', str(0))
df['emp_length_int'] = df['emp_length_int'].str.replace(' years', '')
df['emp_length_int'] = df['emp_length_int'].str.replace(' year', '')
df['emp_length_int'] = pd.to_numeric(df['emp_length_int'])
df['emp_length_int'].unique()

array([10.,  0.,  1.,  3.,  8.,  9.,  4.,  5.,  6.,  2.,  7., nan])

In [13]:
df['earliest_cr_line'].head(3).unique()

array(['Jan-85', 'Apr-99', 'Nov-01'], dtype=object)

In [14]:
df['earliest_cr_line_date'] = pd.to_datetime(df['earliest_cr_line'], format = '%b-%y')
df['mths_since_earliest_cr_line'] = round((pd.to_datetime('2017-12-01') - df['earliest_cr_line_date'])/np.timedelta64(1, 'M'))
df['mths_since_earliest_cr_line'].head(3).unique()

array([395., 224., 193.])

In [15]:
df['mths_since_earliest_cr_line'].describe().head(4)

count   466256.00
mean       239.48
std         93.97
min       -612.00
Name: mths_since_earliest_cr_line, dtype: float64

In [16]:
df['mths_since_earliest_cr_line'][df['mths_since_earliest_cr_line']<0] = df['mths_since_earliest_cr_line'].max()

In [17]:
min(df['mths_since_earliest_cr_line'])

73.0

In [18]:
df['issue_d'].unique()[:3]

array(['Dec-11', 'Nov-11', 'Oct-11'], dtype=object)

In [19]:
df['issue_d_date'] = pd.to_datetime(df['issue_d'], format = '%b-%y')
df['mths_since_issue_d'] = round((pd.to_datetime('2017-12-01') - df['issue_d_date'])/np.timedelta64(1, 'M'))
df['mths_since_issue_d'].unique()[:3]

array([72., 73., 74.])

In [20]:
df.drop(['term', 'emp_length', 'earliest_cr_line', 'earliest_cr_line_date', 'issue_d', 'issue_d_date'], axis=1, inplace=True)
df.head(3)

,funded_amnt,int_rate,installment,grade,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d
0,5000,10.65,162.87,B,RENT,24000.00,Verified,Fully Paid,n,credit_card,AZ,27.65,0.00,1.00,3.00,0.00,9.00,f,0.00,5861.07,5000.00,861.07,0.00,0.00,36,10.00,395.00,72.00
1,2500,15.27,59.83,C,RENT,30000.00,Source Verified,Charged Off,n,car,GA,1.00,0.00,5.00,3.00,0.00,4.00,f,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00
2,2400,15.96,84.33,C,RENT,12252.00,Not Verified,Fully Paid,n,small_business,IL,8.72,0.00,2.00,2.00,0.00,10.00,f,0.00,3003.65,2400.00,603.65,0.00,0.00,36,10.00,193.00,72.00


### 2.b. Fill columns with missing values:<br><br>i) Replace values with 0: emp_length_int, delinq_2yrs, mths_since_earliest_cr_line, inq_last_6mths, open_acc, pub_rec, total_acc, acc_now_delinq<br>ii) Replace values with funded amount: total_rev_hi_lim<br>iii) Replace values with the column average: annual_inc

In [21]:
df.isnull().sum().sum()

21215

In [22]:
df['emp_length_int'].fillna(0, inplace=True)
df['delinq_2yrs'].fillna(0, inplace=True)
df['mths_since_earliest_cr_line'].fillna(0, inplace=True)
df['inq_last_6mths'].fillna(0, inplace=True)
df['open_acc'].fillna(0, inplace=True)
df['pub_rec'].fillna(0, inplace=True)
df['total_acc'].fillna(0, inplace=True)
df['acc_now_delinq'].fillna(0, inplace=True)
df['annual_inc'].fillna(df['annual_inc'].mean(), inplace=True)

In [23]:
df.isnull().sum().sum()

0

### 2.c. Create the target variable from loan status column

In [24]:
df['good_bad'] = np.where(df['loan_status'].isin(['Charged Off', 'Default',
                                                           'Does not meet the credit policy. Status:Charged Off',
                                                           'Late (31-120 days)']), 0, 1)
df.drop(['loan_status'], axis=1, inplace=True)
df.head(3)

,funded_amnt,int_rate,installment,grade,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,good_bad
0,5000,10.65,162.87,B,RENT,24000.00,Verified,n,credit_card,AZ,27.65,0.00,1.00,3.00,0.00,9.00,f,0.00,5861.07,5000.00,861.07,0.00,0.00,36,10.00,395.00,72.00,1
1,2500,15.27,59.83,C,RENT,30000.00,Source Verified,n,car,GA,1.00,0.00,5.00,3.00,0.00,4.00,f,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00,0
2,2400,15.96,84.33,C,RENT,12252.00,Not Verified,n,small_business,IL,8.72,0.00,2.00,2.00,0.00,10.00,f,0.00,3003.65,2400.00,603.65,0.00,0.00,36,10.00,193.00,72.00,1


In [25]:
bad_df = df[df['good_bad']==0].copy()
bad_df.drop(['good_bad'], axis=1, inplace=True)
bad_df.head(3)

,funded_amnt,int_rate,installment,grade,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d
1,2500,15.27,59.83,C,RENT,30000.00,Source Verified,n,car,GA,1.00,0.00,5.00,3.00,0.00,4.00,f,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00
8,5600,21.28,152.39,F,OWN,40000.00,Source Verified,n,small_business,CA,5.55,0.00,2.00,11.00,0.00,13.00,f,0.00,646.02,162.02,294.94,189.06,0.00,60,4.00,164.00,72.00
9,5375,12.69,121.45,B,RENT,15000.00,Verified,n,other,TX,18.08,0.00,0.00,2.00,0.00,3.00,f,0.00,1476.19,673.48,533.42,269.29,0.00,60,0.00,159.00,72.00


# 

## 3) Fine and Coarse Classing using Weight of Evidence (WoE)

In [26]:
def get_woe_matrix(df, col_name, discrete=True):
    woe_mat = df.groupby([col_name, 'good_bad'])[col_name].count().unstack('good_bad')
    woe_mat[0].fillna(0, inplace=True)
    woe_mat[1].fillna(0, inplace=True)
    woe_mat.iloc[:,[0,1]].fillna(0)
    woe_mat['pct_good'] = woe_mat[1]/woe_mat[1].sum()
    woe_mat['pct_bad'] = woe_mat[0]/woe_mat[0].sum()
    woe_mat['num_obs'] = woe_mat[0]+woe_mat[1]
    woe_value = np.log(woe_mat['pct_good']/woe_mat['pct_bad'])
    woe_mat['woe'] = np.where(np.isinf(woe_value), np.nan,
                     np.where(np.isinf(-woe_value), np.nan, woe_value))         
    woe_mat['pct_obs'] = (woe_mat[0]+woe_mat[1])/(woe_mat[0].sum()+woe_mat[1].sum())*100
    woe_mat['info_val'] = ((woe_mat['pct_good']-woe_mat['pct_bad'])*woe_mat['woe']).sum()
    woe_mat = woe_mat.sort_values(by='woe')
    return woe_mat

def plot_woe(woe_mat, col_name, plot_text=True, xtick_rot=0, font_size=9, indices=None):
    if indices:
        ind1, ind2 = indices
        if ind1 is None:
            woe_mat = woe_mat.iloc[:ind2]
        elif ind2 is None:
            woe_mat = woe_mat.iloc[ind1:]
        else:
            woe_mat = woe_mat.iloc[ind1:ind2]
    x = woe_mat.index.values
    y = woe_mat['woe']
    n_cat = len(x)
    plt.figure(figsize=(15,3))
    plt.plot(x, y, 'bo--')
    plt.xlabel(col_name.capitalize())
    plt.ylabel('Weight of Evidence (WoE)')
    plt.title('Weight of Evidence for \''+col_name+'\' column', fontsize=14, weight='bold')
    y_min, y_max = plt.ylim()
    plt.ylim(y_min-(y_max-y_min)*0.1, y_max+(y_max-y_min)*0.1)
    plt.xticks(rotation=xtick_rot)
    if plot_text:
        for index in range(len(x)):
            if y[index] >= 0:
                plt.text(x[index], y[index], round(y[index],3), ha='center', va='bottom', color='green', fontsize=font_size, weight='bold')
                plt.text(x[index], y[index], str(round(woe_mat['pct_obs'][index],1))+'%', ha='center', va='top', color='blue', fontsize=font_size)
            else:
                plt.text(x[index], y[index], round(y[index],3), ha='center', va='bottom', color='red', fontsize=font_size, weight='bold')
                plt.text(x[index], y[index], str(round(woe_mat['pct_obs'][index],1))+'%', ha='center', va='top', color='blue', fontsize=font_size)
    return

In [27]:
direct_dummies = ['pymnt_plan', 'initial_list_status', 'verification_status', 'grade']

In [28]:
for col_name in direct_dummies:
    for unique_val in sorted(bad_df[col_name].unique()):
        bad_df[col_name+' : '+str(unique_val)] = (bad_df[col_name]==unique_val).astype(int)
bad_df.head(3)

,funded_amnt,int_rate,installment,grade,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,pymnt_plan : n,pymnt_plan : y,initial_list_status : f,initial_list_status : w,verification_status : Not Verified,verification_status : Source Verified,verification_status : Verified,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,grade : G
1,2500,15.27,59.83,C,RENT,30000.00,Source Verified,n,car,GA,1.00,0.00,5.00,3.00,0.00,4.00,f,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00,1,0,1,0,0,1,0,0,0,1,0,0,0,0
8,5600,21.28,152.39,F,OWN,40000.00,Source Verified,n,small_business,CA,5.55,0.00,2.00,11.00,0.00,13.00,f,0.00,646.02,162.02,294.94,189.06,0.00,60,4.00,164.00,72.00,1,0,1,0,0,1,0,0,0,0,0,0,1,0
9,5375,12.69,121.45,B,RENT,15000.00,Verified,n,other,TX,18.08,0.00,0.00,2.00,0.00,3.00,f,0.00,1476.19,673.48,533.42,269.29,0.00,60,0.00,159.00,72.00,1,0,1,0,0,0,1,0,1,0,0,0,0,0


In [29]:
grouped_dummies = ['home_ownership', 'purpose', 'addr_state']

In [30]:
home_ownership__classes = ' other none rent; own; mortgage any'
purpose__classes = ' small_business educational; renewable_energy moving house other; '\
                  +'medical wedding vacation debt_consolidation; home_improvement major_purchase car credit_card'
addr_state__classes = ' NE IA NV; HI FL AL; NY LA; NM OK NC MO MD NJ VA; CA; AZ MI UT TN AR; PA RI; OH KY DE MN SD MA IN; '\
                     +'GA WA VM OR; TX; IL CT MT; CO SC; KS VT AK MS ID NH; WV WY DC ME'

for group_label in home_ownership__classes.split(';'):
    bad_df['home_ownership : '+group_label[1:]] = bad_df['home_ownership'].isin(group_label.split(' ')[1:]).astype(int)

for group_label in purpose__classes.split(';'):
    bad_df['purpose : '+group_label[1:]] = bad_df['purpose'].isin(group_label.split(' ')[1:]).astype(int)

for group_label in addr_state__classes.split(';'):
    bad_df['addr_state : '+group_label[1:]] = bad_df['addr_state'].isin(group_label.split(' ')[1:]).astype(int)

bad_df.head(3)

,funded_amnt,int_rate,installment,grade,home_ownership,annual_inc,verification_status,pymnt_plan,purpose,addr_state,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,initial_list_status,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,pymnt_plan : n,pymnt_plan : y,initial_list_status : f,initial_list_status : w,verification_status : Not Verified,verification_status : Source Verified,verification_status : Verified,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,grade : G,home_ownership : other none rent,home_ownership : own,home_ownership : mortgage any,purpose : small_business educational,purpose : renewable_energy moving house other,purpose : medical wedding vacation debt_consolidation,purpose : home_improvement major_purchase car credit_card,addr_state : NE IA NV,addr_state : HI FL AL,addr_state : NY LA,addr_state : NM OK NC MO MD NJ VA,addr_state : CA,addr_state : AZ MI UT TN AR,addr_state : PA RI,addr_state : OH KY DE MN SD MA IN,addr_state : GA WA VM OR,addr_state : TX,addr_state : IL CT MT,addr_state : CO SC,addr_state : KS VT AK MS ID NH,addr_state : WV WY DC ME
1,2500,15.27,59.83,C,RENT,30000.00,Source Verified,n,car,GA,1.00,0.00,5.00,3.00,0.00,4.00,f,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,5600,21.28,152.39,F,OWN,40000.00,Source Verified,n,small_business,CA,5.55,0.00,2.00,11.00,0.00,13.00,f,0.00,646.02,162.02,294.94,189.06,0.00,60,4.00,164.00,72.00,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,5375,12.69,121.45,B,RENT,15000.00,Verified,n,other,TX,18.08,0.00,0.00,2.00,0.00,3.00,f,0.00,1476.19,673.48,533.42,269.29,0.00,60,0.00,159.00,72.00,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [31]:
bad_df.drop(direct_dummies+grouped_dummies, axis=1, inplace=True)
bad_df.head(3)

,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,pymnt_plan : n,pymnt_plan : y,initial_list_status : f,initial_list_status : w,verification_status : Not Verified,verification_status : Source Verified,verification_status : Verified,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,grade : G,home_ownership : other none rent,home_ownership : own,home_ownership : mortgage any,purpose : small_business educational,purpose : renewable_energy moving house other,purpose : medical wedding vacation debt_consolidation,purpose : home_improvement major_purchase car credit_card,addr_state : NE IA NV,addr_state : HI FL AL,addr_state : NY LA,addr_state : NM OK NC MO MD NJ VA,addr_state : CA,addr_state : AZ MI UT TN AR,addr_state : PA RI,addr_state : OH KY DE MN SD MA IN,addr_state : GA WA VM OR,addr_state : TX,addr_state : IL CT MT,addr_state : CO SC,addr_state : KS VT AK MS ID NH,addr_state : WV WY DC ME
1,2500,15.27,59.83,30000.00,1.00,0.00,5.00,3.00,0.00,4.00,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
8,5600,21.28,152.39,40000.00,5.55,0.00,2.00,11.00,0.00,13.00,0.00,646.02,162.02,294.94,189.06,0.00,60,4.00,164.00,72.00,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,5375,12.69,121.45,15000.00,18.08,0.00,0.00,2.00,0.00,3.00,0.00,1476.19,673.48,533.42,269.29,0.00,60,0.00,159.00,72.00,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [32]:
ref_columns = 'pymnt_plan : y, initial_list_status : f, verification_status : Verified, '+\
              'home_ownership : other none rent, grade : G, purpose : small_business educational, addr_state : NE IA NV'
ref_columns = ref_columns.split(', ')

In [33]:
bad_df.drop(ref_columns, axis=1, inplace=True)
bad_df.head(3)

,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,pymnt_plan : n,initial_list_status : w,verification_status : Not Verified,verification_status : Source Verified,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,home_ownership : own,home_ownership : mortgage any,purpose : renewable_energy moving house other,purpose : medical wedding vacation debt_consolidation,purpose : home_improvement major_purchase car credit_card,addr_state : HI FL AL,addr_state : NY LA,addr_state : NM OK NC MO MD NJ VA,addr_state : CA,addr_state : AZ MI UT TN AR,addr_state : PA RI,addr_state : OH KY DE MN SD MA IN,addr_state : GA WA VM OR,addr_state : TX,addr_state : IL CT MT,addr_state : CO SC,addr_state : KS VT AK MS ID NH,addr_state : WV WY DC ME
1,2500,15.27,59.83,30000.00,1.00,0.00,5.00,3.00,0.00,4.00,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0
8,5600,21.28,152.39,40000.00,5.55,0.00,2.00,11.00,0.00,13.00,0.00,646.02,162.02,294.94,189.06,0.00,60,4.00,164.00,72.00,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,5375,12.69,121.45,15000.00,18.08,0.00,0.00,2.00,0.00,3.00,0.00,1476.19,673.48,533.42,269.29,0.00,60,0.00,159.00,72.00,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


# 

## 4) Train-test split and Beta Regression

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [35]:
X = bad_df.drop(['recoveries'], axis=1).copy()
y = bad_df['recoveries'].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [36]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [37]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [38]:
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

In [39]:
bad_df['LGD'] = model.predict(scaler.transform(X))/bad_df['funded_amnt']*100
bad_df.head(3)

,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,pymnt_plan : n,initial_list_status : w,verification_status : Not Verified,verification_status : Source Verified,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,home_ownership : own,home_ownership : mortgage any,purpose : renewable_energy moving house other,purpose : medical wedding vacation debt_consolidation,purpose : home_improvement major_purchase car credit_card,addr_state : HI FL AL,addr_state : NY LA,addr_state : NM OK NC MO MD NJ VA,addr_state : CA,addr_state : AZ MI UT TN AR,addr_state : PA RI,addr_state : OH KY DE MN SD MA IN,addr_state : GA WA VM OR,addr_state : TX,addr_state : IL CT MT,addr_state : CO SC,addr_state : KS VT AK MS ID NH,addr_state : WV WY DC ME,LGD
1,2500,15.27,59.83,30000.00,1.00,0.00,5.00,3.00,0.00,4.00,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,4.60
8,5600,21.28,152.39,40000.00,5.55,0.00,2.00,11.00,0.00,13.00,0.00,646.02,162.02,294.94,189.06,0.00,60,4.00,164.00,72.00,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.34
9,5375,12.69,121.45,15000.00,18.08,0.00,0.00,2.00,0.00,3.00,0.00,1476.19,673.48,533.42,269.29,0.00,60,0.00,159.00,72.00,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,4.99


In [40]:
bad_df['LGD'] = np.where(bad_df['LGD']<0, 0, bad_df['LGD'])
bad_df['LGD'] = np.where(bad_df['LGD']>100, 100, bad_df['LGD'])
bad_df.head(3)

,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,pymnt_plan : n,initial_list_status : w,verification_status : Not Verified,verification_status : Source Verified,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,home_ownership : own,home_ownership : mortgage any,purpose : renewable_energy moving house other,purpose : medical wedding vacation debt_consolidation,purpose : home_improvement major_purchase car credit_card,addr_state : HI FL AL,addr_state : NY LA,addr_state : NM OK NC MO MD NJ VA,addr_state : CA,addr_state : AZ MI UT TN AR,addr_state : PA RI,addr_state : OH KY DE MN SD MA IN,addr_state : GA WA VM OR,addr_state : TX,addr_state : IL CT MT,addr_state : CO SC,addr_state : KS VT AK MS ID NH,addr_state : WV WY DC ME,LGD
1,2500,15.27,59.83,30000.00,1.00,0.00,5.00,3.00,0.00,4.00,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,4.60
8,5600,21.28,152.39,40000.00,5.55,0.00,2.00,11.00,0.00,13.00,0.00,646.02,162.02,294.94,189.06,0.00,60,4.00,164.00,72.00,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.34
9,5375,12.69,121.45,15000.00,18.08,0.00,0.00,2.00,0.00,3.00,0.00,1476.19,673.48,533.42,269.29,0.00,60,0.00,159.00,72.00,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,4.99


In [41]:
bad_df['EAD'] = bad_df['funded_amnt']*(100-bad_df['LGD'])/100
bad_df.head(3)

,funded_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,total_acc,out_prncp,total_pymnt,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,term_int,emp_length_int,mths_since_earliest_cr_line,mths_since_issue_d,pymnt_plan : n,initial_list_status : w,verification_status : Not Verified,verification_status : Source Verified,grade : A,grade : B,grade : C,grade : D,grade : E,grade : F,home_ownership : own,home_ownership : mortgage any,purpose : renewable_energy moving house other,purpose : medical wedding vacation debt_consolidation,purpose : home_improvement major_purchase car credit_card,addr_state : HI FL AL,addr_state : NY LA,addr_state : NM OK NC MO MD NJ VA,addr_state : CA,addr_state : AZ MI UT TN AR,addr_state : PA RI,addr_state : OH KY DE MN SD MA IN,addr_state : GA WA VM OR,addr_state : TX,addr_state : IL CT MT,addr_state : CO SC,addr_state : KS VT AK MS ID NH,addr_state : WV WY DC ME,LGD,EAD
1,2500,15.27,59.83,30000.00,1.00,0.00,5.00,3.00,0.00,4.00,0.00,1008.71,456.46,435.17,117.08,0.00,60,0.00,224.00,72.00,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,4.60,2384.99
8,5600,21.28,152.39,40000.00,5.55,0.00,2.00,11.00,0.00,13.00,0.00,646.02,162.02,294.94,189.06,0.00,60,4.00,164.00,72.00,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.34,5412.77
9,5375,12.69,121.45,15000.00,18.08,0.00,0.00,2.00,0.00,3.00,0.00,1476.19,673.48,533.42,269.29,0.00,60,0.00,159.00,72.00,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,4.99,5106.56


In [42]:
bad_df['EAD'].sum()/df['funded_amnt'].sum()*100

10.531587166843396

### The bank is experiencing 10.53% losses on funded loans, which is a bit larger than the capital requirement; hence some changes in loan funding strategies should be undertaken.